Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import zipfile
from matplotlib import pyplot as plt
import cv2
import random
import keras
import tensorflow as tf
from keras.models import Model, load_model
import shutil


Print TensorFlow and Keras versions to ensure compatibility

In [ ]:

print(tf.__version__)
print(keras.__version__)


Change Working Directory (Adjust this as per your local path)

In [ ]:

os.chdir("<your project directory>/Classifier/")
print(os.getcwd())


Define Directories and Load the Model

In [ ]:
Dir_Name = '<directory name>'
Root_Dir = "<your project directory>/Classifier/Model-Training/Data/nf/"
mon = Dir_Name

# Load the pre-trained model for classification
try:
    model_path = "<your project directory>/Classifier/Model-Training/Code/ModelForClassifier/CNN.model7"
    new_model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")


Create and Change to a New Directory for Storing Classified Images

In [ ]:
new_directory = os.path.join(Root_Dir, mon)
if not os.path.exists(new_directory):
    os.mkdir(new_directory)
print(f"Directory is set to: {new_directory}")

os.chdir(new_directory)
print(f"Current working directory: {os.getcwd()}")


Unzip the Data Files to the New Directory

In [ ]:
def unzip_data(zip_path, extract_to):
    """
    Unzip the dataset into the specified directory, avoiding nested folders.
    
    Parameters:
    zip_path (str): Path to the zip file.
    extract_to (str): Directory to extract the zip file to.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Data unzipped successfully to {extract_to}")

# Unzip the dataset into the new directory
zip_file_path = os.path.join(Root_Dir, Dir_Name + '.zip')
if os.path.exists(zip_file_path):
    unzip_data(zip_file_path, new_directory)
else:
    print(f"Zip file not found: {zip_file_path}")

print(os.listdir(new_directory))


Prepare Image Paths for Prediction

In [ ]:
# Get a dictionary of folders and their respective image names as key-value pairs
folder_filenames_dict = {}
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(new_directory)):
    split_dirname_list = dirpath.split(os.sep)
    if i == 0:
        root_folder_name = dirpath
        telegram_channels_names = dirnames
    elif i == 2:
        key = split_dirname_list[-2]
        value = [f for f in filenames if '.ini' != os.path.splitext(f)[1]]
        folder_filenames_dict[key] = [len(value), value]
    elif split_dirname_list[-1] == "photos":
        key = split_dirname_list[-2]
        value = filenames
        folder_filenames_dict[key] = [len(filenames), value]

print(folder_filenames_dict)


In [ ]:
# Function to prepare images for prediction
def prepare_image(filepath):
    """
    Prepare an image for prediction by resizing and normalizing.
    
    Parameters:
    filepath (str): Path to the image file.
    
    Returns:
    np.array: Reshaped and normalized image array.
    """
    img_size = 224
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    rgb_img = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    resized_img = cv2.resize(rgb_img, (img_size, img_size))
    normalized_img = resized_img / 255.0
    return normalized_img.reshape(-1, img_size, img_size, 3)

# %%
print("Image preparation function defined.")


In [ ]:
# Prepare file paths for images to be predicted
pred_class_list = []
original_class_list = []
photos_folder = 'photos'
image_paths = []

# Construct image paths
for folder_name, (count, filenames) in folder_filenames_dict.items():
    for filename in filenames:
        image_path = os.path.join(new_directory, folder_name, photos_folder, filename)
        image_paths.append(image_path)

# Remove the first element if necessary (example adjustment)
image_paths = image_paths[1:]

# Print the constructed image paths
print(image_paths)
print(f"Total images: {len(image_paths)}")


In [ ]:
# Predict a sample image to verify the model's functionality
from skimage.io import imread
from PIL import Image as Ima

# Verify the first image path and predict
if image_paths:
    first_image_path = image_paths[0]
    if os.path.exists(first_image_path):
        img = imread(first_image_path)
        data = Ima.fromarray(img)
        data.show()

        prediction = new_model.predict([prepare_image(first_image_path)])
        predicted_class = np.argmax(prediction)
        print(f"Predicted class for the first image: {predicted_class}")
    else:
        print(f"Image not found: {first_image_path}")
else:
    print("No images found for prediction.")


Predict and Classify Images

In [ ]:
from tqdm import tqdm

cheques_class_images = []
bank_acc_class_images = []
money_apps_class_images = []
error_images = []

# Predict and classify images
for image_path in tqdm(image_paths):
    try:
        prediction = new_model.predict([prepare_image(image_path)])
        predicted_class = np.argmax(prediction)
        
        if predicted_class == 1:
            cheques_class_images.append(image_path)
        elif predicted_class == 0:
            bank_acc_class_images.append(image_path)
        elif predicted_class == 2:
            money_apps_class_images.append(image_path)
    except Exception as e:
        error_images.append((image_path, str(e)))

print(f"Total errors: {len(error_images)}")
print(f"Cheques class images: {len(cheques_class_images)}")
print(f"Bank account class images: {len(bank_acc_class_images)}")
print(f"Money apps class images: {len(money_apps_class_images)}")


In [ ]:
# Save the lists of file paths for future reference
def save_list_to_file(list_data, filename):
    """
    Save a list of strings to a file.
    
    Parameters:
    list_data (list): List of strings to be saved.
    filename (str): Name of the file to save the data.
    """
    with open(filename, 'w') as file:
        for item in list_data:
            file.write(f"{item}\n")
    print(f"Data saved to {filename}")

# Save classified image paths
mon_d1_d2 = mon
save_list_to_file(bank_acc_class_images, f"{mon_d1_d2}_bankAccClassImages.txt")
save_list_to_file(money_apps_class_images, f"{mon_d1_d2}_moneyAppsClassImages.txt")
save_list_to_file(cheques_class_images, f"{mon_d1_d2}_chequesClassImages.txt")


In [ ]:
# Copy classified images to respective folders
def copy_images_to_folder(image_list, destination_folder):
    """
    Copy images from a list to a specified folder.
    
    Parameters:
    image_list (list): List of image paths to be copied.
    destination_folder (str): Destination folder path.
    """
    if not os.path.exists(destination_folder):
        os.mkdir(destination_folder)

    for image_path in image_list:
        source = image_path
        if os.path.exists(source):
            shutil.copy2(source, destination_folder)
        else:
            print(f"Source image not found: {source}")
    print(f"Copied {len(image_list)} images to {destination_folder}")

cheques_folder = os.path.join(new_directory, f"{mon_d1_d2}_chequesClassified")
money_apps_folder = os.path.join(new_directory, f"{mon_d1_d2}_moneyAppsClassified")
bank_acc_folder = os.path.join(new_directory, f"{mon_d1_d2}_bankAccClassified")

copy_images_to_folder(cheques_class_images, cheques_folder)
copy_images_to_folder(money_apps_class_images, money_apps_folder)
copy_images_to_folder(bank_acc_class_images, bank_acc_folder)


In [ ]:
# Create ZIP archives of the classified folders
def create_zip_archive(folder_path):
    """
    Create a ZIP archive of a specified folder.
    
    Parameters:
    folder_path (str): Path of the folder to be archived.
    """
    archive_path = f"{folder_path}.zip"
    shutil.make_archive(folder_path, 'zip', folder_path)
    print(f"Created ZIP archive: {archive_path}")

create_zip_archive(cheques_folder)
create_zip_archive(money_apps_folder)
create_zip_archive(bank_acc_folder)
